# 05_probability_distributions.ipynb

## From First Principles: Probability Distributions and Their Properties

This notebook explores fundamental probability distributions, their properties, and applications in machine learning. We'll implement key distributions from scratch and visualize their characteristics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma, factorial
import math
%matplotlib inline

## 1. Discrete Probability Distributions

Discrete distributions model outcomes that take on distinct, countable values.

### Bernoulli Distribution

The Bernoulli distribution models a single trial with two possible outcomes (success/failure).

In [ ]:
def bernoulli_pmf(k, p):
    """
    Probability mass function for Bernoulli distribution.
    
    Args:
        k: Outcome (0 or 1)
        p: Probability of success
        
    Returns:
        Probability of outcome k
    """
    if k == 1:
        return p
    elif k == 0:
        return 1 - p
    else:
        return 0

def bernoulli_sample(p, size=1):
    """Generate samples from Bernoulli distribution"""
    return (np.random.random(size) < p).astype(int)

# Example: Fair coin flip (p=0.5)
p = 0.5
k_values = [0, 1]
pmf_values = [bernoulli_pmf(k, p) for k in k_values]

print(f"Bernoulli distribution (p={p}):")
for k, prob in zip(k_values, pmf_values):
    print(f"P(X = {k}) = {prob}")

# Generate samples
samples = bernoulli_sample(p, size=1000)
print(f"Sample mean: {np.mean(samples):.3f} (expected: {p})")

# Plot
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(k_values, pmf_values, tick_label=k_values)
plt.title(f'Bernoulli PMF (p={p})')
plt.xlabel('Outcome')
plt.ylabel('Probability')

plt.subplot(1, 2, 2)
plt.hist(samples, bins=np.arange(3)-0.5, density=True, alpha=0.7, edgecolor='black')
plt.title(f'Bernoulli Samples (n=1000)')
plt.xlabel('Outcome')
plt.ylabel('Frequency')
plt.xticks([0, 1])

plt.tight_layout()
plt.show()

### Binomial Distribution

The binomial distribution models the number of successes in n independent Bernoulli trials.

In [ ]:
def binomial_coefficient(n, k):
    """Calculate binomial coefficient 'n choose k'"""
    if k > n or k < 0:
        return 0
    if k == 0 or k == n:
        return 1
    
    # Use the more efficient calculation to avoid large factorials
    result = 1
    for i in range(min(k, n - k)):
        result = result * (n - i) // (i + 1)
    return result

def binomial_pmf(k, n, p):
    """
    Probability mass function for binomial distribution.
    
    Args:
        k: Number of successes
        n: Number of trials
        p: Probability of success in each trial
        
    Returns:
        Probability of k successes in n trials
    """
    if k < 0 or k > n:
        return 0
    
    coeff = binomial_coefficient(n, k)
    return coeff * (p ** k) * ((1 - p) ** (n - k))

def binomial_sample(n, p, size=1):
    """Generate samples from binomial distribution"""
    return np.sum(np.random.random((size, n)) < p, axis=1)

# Example: 10 coin flips, p=0.3
n = 10
p = 0.3
k_values = list(range(n + 1))
pmf_values = [binomial_pmf(k, n, p) for k in k_values]

print(f"Binomial distribution (n={n}, p={p}):")
print(f"Mean: {n*p}, Variance: {n*p*(1-p)}")

# Generate samples
samples = binomial_sample(n, p, size=1000)
print(f"Sample mean: {np.mean(samples):.3f} (expected: {n*p})")
print(f"Sample var: {np.var(samples):.3f} (expected: {n*p*(1-p)})")

# Plot
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(k_values, pmf_values)
plt.title(f'Binomial PMF (n={n}, p={p})')
plt.xlabel('Number of Successes')
plt.ylabel('Probability')

plt.subplot(1, 2, 2)
plt.hist(samples, bins=np.arange(n+2)-0.5, density=True, alpha=0.7, edgecolor='black')
plt.title(f'Binomial Samples (n=1000)')
plt.xlabel('Number of Successes')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## 2. Continuous Probability Distributions

Continuous distributions model outcomes that can take any value in a continuous range.

### Normal (Gaussian) Distribution

The normal distribution is fundamental in statistics and machine learning due to the Central Limit Theorem.

In [ ]:
def normal_pdf(x, mu=0, sigma=1):
    """
    Probability density function for normal distribution.
    
    Args:
        x: Point at which to evaluate the PDF
        mu: Mean
        sigma: Standard deviation
        
    Returns:
        PDF value at x
    """
    coefficient = 1 / (sigma * np.sqrt(2 * np.pi))
    exponent = -0.5 * ((x - mu) / sigma) ** 2
    return coefficient * np.exp(exponent)

def normal_sample(mu=0, sigma=1, size=1):
    """Generate samples from normal distribution using Box-Muller transform"""
    # Using numpy's random for simplicity, but could implement Box-Muller
    return np.random.normal(mu, sigma, size)

# Example: Normal distribution with μ=0, σ=1
mu, sigma = 0, 1
x = np.linspace(-4, 4, 1000)
pdf_values = normal_pdf(x, mu, sigma)

print(f"Normal distribution (μ={mu}, σ={sigma}):")
print(f"Mean: {mu}, Variance: {sigma**2}")

# Generate samples
samples = normal_sample(mu, sigma, size=1000)
print(f"Sample mean: {np.mean(samples):.3f} (expected: {mu})")
print(f"Sample std: {np.std(samples):.3f} (expected: {sigma})")

# Plot
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(x, pdf_values, linewidth=2)
plt.title(f'Normal PDF (μ={mu}, σ={sigma})')
plt.xlabel('x')
plt.ylabel('Density')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(samples, bins=50, density=True, alpha=0.7, edgecolor='black')
plt.plot(x, pdf_values, 'r-', linewidth=2, label='True PDF')
plt.title(f'Normal Samples vs True PDF (n=1000)')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Exponential Distribution

The exponential distribution models the time between events in a Poisson process.

In [ ]:
def exponential_pdf(x, lambd):
    """
    Probability density function for exponential distribution.
    
    Args:
        x: Point at which to evaluate the PDF (x >= 0)
        lambd: Rate parameter (lambda > 0)
        
    Returns:
        PDF value at x
    """
    if x < 0:
        return 0
    return lambd * np.exp(-lambd * x)

def exponential_sample(lambd, size=1):
    """Generate samples from exponential distribution using inverse transform sampling"""
    u = np.random.uniform(0, 1, size)
    return -np.log(1 - u) / lambd

# Example: Exponential distribution with λ=0.5
lambd = 0.5
x = np.linspace(0, 10, 1000)
pdf_values = [exponential_pdf(xi, lambd) for xi in x]

print(f"Exponential distribution (λ={lambd}):")
print(f"Mean: {1/lambd}, Variance: {1/lambd**2}")

# Generate samples
samples = exponential_sample(lambd, size=1000)
print(f"Sample mean: {np.mean(samples):.3f} (expected: {1/lambd})")
print(f"Sample var: {np.var(samples):.3f} (expected: {1/lambd**2})")

# Plot
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(x, pdf_values, linewidth=2)
plt.title(f'Exponential PDF (λ={lambd})')
plt.xlabel('x')
plt.ylabel('Density')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(samples, bins=50, density=True, alpha=0.7, edgecolor='black')
plt.plot(x, pdf_values, 'r-', linewidth=2, label='True PDF')
plt.title(f'Exponential Samples vs True PDF (n=1000)')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Multivariate Distributions

Multivariate distributions model multiple random variables simultaneously.

### Multivariate Normal Distribution

The multivariate normal distribution is the generalization of the univariate normal to multiple dimensions.

In [ ]:
def multivariate_normal_pdf(x, mu, sigma):
    """
    Probability density function for multivariate normal distribution.
    
    Args:
        x: Point at which to evaluate the PDF (array-like)
        mu: Mean vector
        sigma: Covariance matrix
        
    Returns:
        PDF value at x
    """
    x = np.asarray(x)
    mu = np.asarray(mu)
    sigma = np.asarray(sigma)
    
    if x.shape != mu.shape:
        raise ValueError("x and mu must have the same shape")
    
    d = len(mu)  # dimension
    
    # Calculate determinant and inverse of covariance matrix
    det_sigma = np.linalg.det(sigma)
    if det_sigma <= 0:
        raise ValueError("Covariance matrix must be positive definite")
    
    inv_sigma = np.linalg.inv(sigma)
    
    # Calculate the exponent
    diff = x - mu
    exponent = -0.5 * diff.T @ inv_sigma @ diff
    
    # Calculate the normalization constant
    norm_const = 1.0 / np.sqrt((2 * np.pi) ** d * det_sigma)
    
    return norm_const * np.exp(exponent)

def multivariate_normal_sample(mu, sigma, size=1):
    """Generate samples from multivariate normal distribution"""
    return np.random.multivariate_normal(mu, sigma, size)

# Example: 2D multivariate normal
mu = np.array([0, 1])
sigma = np.array([[1, 0.5],
                  [0.5, 1]])

# Generate samples
samples = multivariate_normal_sample(mu, sigma, size=1000)

print(f"2D Multivariate Normal (μ=[{mu[0]}, {mu[1]}]):")
print(f"Covariance matrix:\n{sigma}")
print(f"Sample mean: [{np.mean(samples[:, 0]):.3f}, {np.mean(samples[:, 1]):.3f}] (expected: [{mu[0]}, {mu[1]}])")
print(f"Sample covariance:\n{np.cov(samples.T)}")

# Create grid for contour plot
x = np.linspace(-4, 4, 100)
y = np.linspace(-3, 5, 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))

# Calculate PDF values
Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = multivariate_normal_pdf(pos[i, j], mu, sigma)

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.contour(X, Y, Z, levels=10)
plt.scatter(samples[:, 0], samples[:, 1], alpha=0.6, s=10)
plt.title('2D Multivariate Normal Samples + Contours')
plt.xlabel('X1')
plt.ylabel('X2')

plt.subplot(1, 2, 2)
plt.scatter(samples[:, 0], samples[:, 1], alpha=0.6)
plt.title('2D Multivariate Normal Samples')
plt.xlabel('X1')
plt.ylabel('X2')

plt.tight_layout()
plt.show()

## 4. Distribution Fitting and Parameter Estimation

Maximum Likelihood Estimation (MLE) is a common approach to estimate distribution parameters from data.

In [ ]:
def mle_normal_params(data):
    """
    Calculate MLE parameters for normal distribution.
    For normal distribution, MLE estimates are simply the sample mean and variance.
    """
    mu_mle = np.mean(data)
    sigma_mle = np.sqrt(np.mean((data - mu_mle)**2))  # biased estimator
    return mu_mle, sigma_mle

def mle_exponential_param(data):
    """
    Calculate MLE parameter for exponential distribution.
    For exponential distribution, MLE estimate is 1 / sample_mean.
    """
    return 1 / np.mean(data)

# Test MLE estimation
print("Testing MLE parameter estimation:")

# Normal distribution
true_mu, true_sigma = 2, 1.5
normal_data = np.random.normal(true_mu, true_sigma, 1000)
est_mu, est_sigma = mle_normal_params(normal_data)

print(f"Normal distribution:")
print(f"  True params: μ={true_mu}, σ={true_sigma}")
print(f"  Estimated: μ={est_mu:.3f}, σ={est_sigma:.3f}")

# Exponential distribution
true_lambda = 0.8
exp_data = np.random.exponential(1/true_lambda, 1000)  # numpy uses scale = 1/lambda
est_lambda = mle_exponential_param(exp_data)

print(f"\nExponential distribution:")
print(f"  True λ: {true_lambda}")
print(f"  Estimated λ: {est_lambda:.3f}")

# Visualize the fitted distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(normal_data, bins=50, density=True, alpha=0.7, label='Data')
x = np.linspace(normal_data.min(), normal_data.max(), 1000)
plt.plot(x, normal_pdf(x, est_mu, est_sigma), 'r-', linewidth=2, label=f'Fitted N({est_mu:.2f}, {est_sigma:.2f})')
plt.title('Normal Distribution Fitting')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(exp_data, bins=50, density=True, alpha=0.7, label='Data')
x = np.linspace(0, exp_data.max(), 1000)
exp_pdf_vals = [exponential_pdf(xi, est_lambda) for xi in x]
plt.plot(x, exp_pdf_vals, 'r-', linewidth=2, label=f'Fitted Exp(λ={est_lambda:.2f})')
plt.title('Exponential Distribution Fitting')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Applications in Machine Learning

Probability distributions are fundamental to many ML algorithms.

In [ ]:
# Example: Naive Bayes classifier using normal distribution assumption

class NaiveBayesNormal:
    """Naive Bayes classifier with normal distribution assumption for features"""
    
    def __init__(self):
        self.classes = None
        self.class_priors = {}
        self.feature_params = {}  # Store (mean, std) for each feature and class
    
    def fit(self, X, y):
        """Fit the Naive Bayes model"""
        self.classes = np.unique(y)
        n_samples = len(y)
        
        for c in self.classes:
            # Calculate class prior
            class_samples = X[y == c]
            self.class_priors[c] = len(class_samples) / n_samples
            
            # Calculate feature parameters (mean, std) for each feature
            self.feature_params[c] = {
                'means': np.mean(class_samples, axis=0),
                'stds': np.std(class_samples, axis=0) + 1e-6  # Add small value to avoid division by zero
            }
    
    def _calculate_likelihood(self, x, mean, std):
        """Calculate likelihood using normal PDF"""
        # Avoid division by zero
        std = np.maximum(std, 1e-6)
        exponent = -0.5 * ((x - mean) / std) ** 2
        coefficient = 1 / (std * np.sqrt(2 * np.pi))
        return coefficient * np.exp(exponent)
    
    def predict_proba(self, X):
        """Predict class probabilities"""
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        probas = np.zeros((n_samples, n_classes))
        
        for i, c in enumerate(self.classes):
            # Calculate class prior
            log_prior = np.log(self.class_priors[c])
            
            # Calculate log likelihood for each feature
            means = self.feature_params[c]['means']
            stds = self.feature_params[c]['stds']
            
            # Calculate log likelihood for all samples and features
            log_likelihood = np.sum(
                np.log(self._calculate_likelihood(X, means, stds)),
                axis=1
            )
            
            # Posterior is proportional to prior * likelihood
            probas[:, i] = log_prior + log_likelihood
        
        # Convert log probabilities to probabilities and normalize
        # Subtract max for numerical stability
        probas = probas - np.max(probas, axis=1, keepdims=True)
        probas = np.exp(probas)
        probas = probas / np.sum(probas, axis=1, keepdims=True)
        
        return probas
    
    def predict(self, X):
        """Predict classes"""
        probas = self.predict_proba(X)
        return self.classes[np.argmax(probas, axis=1)]

# Generate sample data for demonstration
np.random.seed(42)
n_samples = 300

# Class 0: centered at [2, 2]
class0 = np.random.multivariate_normal([2, 2], [[1, 0.3], [0.3, 1]], n_samples//2)
labels0 = np.zeros(n_samples//2)

# Class 1: centered at [0, 0]
class1 = np.random.multivariate_normal([0, 0], [[1, -0.2], [-0.2, 1]], n_samples//2)
labels1 = np.ones(n_samples//2)

# Combine data
X = np.vstack([class0, class1])
y = np.hstack([labels0, labels1])

# Split into train and test
split_idx = int(0.7 * n_samples)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Train the model
nb_model = NaiveBayesNormal()
nb_model.fit(X_train, y_train)

# Make predictions
y_pred = nb_model.predict(X_test)
accuracy = np.mean(y_pred == y_test)

print(f"Naive Bayes Classifier Results:")
print(f"Accuracy: {accuracy:.3f}")

# Visualize results
plt.figure(figsize=(15, 5))

# Plot 1: Training data
plt.subplot(1, 3, 1)
scatter = plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='viridis', alpha=0.7)
plt.title('Training Data')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(scatter)

# Plot 2: Test data with true labels
plt.subplot(1, 3, 2)
scatter = plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='viridis', alpha=0.7)
plt.title('Test Data (True Labels)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(scatter)

# Plot 3: Test data with predicted labels
plt.subplot(1, 3, 3)
scatter = plt.scatter(X_test[:, 0], X_test[:, 1], c=y_pred, cmap='viridis', alpha=0.7)
plt.title(f'Test Data (Predictions, Acc: {accuracy:.3f})')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(scatter)

plt.tight_layout()
plt.show()

## Summary

In this notebook, we've explored:
1. Discrete distributions: Bernoulli, Binomial
2. Continuous distributions: Normal, Exponential
3. Multivariate distributions: Multivariate Normal
4. Parameter estimation: Maximum Likelihood Estimation
5. Applications: Naive Bayes classifier

Understanding probability distributions is crucial for machine learning as they form the basis for many algorithms, from simple classifiers to complex generative models. They help us model uncertainty, make predictions, and understand the underlying patterns in data.